In [ ]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

# Ensure correct working directory
SCRIPT_DIR = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(SCRIPT_DIR, ".."))
sys.path.append(PROJECT_ROOT)  # Ensure imports work

# Import models
from models.nbeatsx import NBeatsX
from models.ctts import CTTS
from models.tft import TemporalFusionTransformer
from models.deepar import DeepAR



ImportError: cannot import name 'prepare_data' from 'utils.preprocessing' (/Users/jamesliu/Documents/GitHub/bloomberg-time-series/utils/preprocessing.py)

In [ ]:

from utils.preprocessing import prepare_data  # Data preparation function
from training.hyperparameters import HPARAMS  # Load hyperparameters
from data.synthetic_data import generate_multiple_series  # Synthetic data

# Set device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generate synthetic data
synthetic_data = generate_multiple_series(num_series=1, T=252)  # 1 year of data
train_data = synthetic_data["synthetic_series_1"]
X_train, Y_train = prepare_data(train_data)

# Create DataLoader
dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(Y_train, dtype=torch.float32))
train_loader = DataLoader(dataset, batch_size=HPARAMS["batch_size"], shuffle=True)

# Define models
models = {
    "nbeatsx": NBeatsX(input_size=HPARAMS["input_size"], output_size=1).to(DEVICE),
    "ctts": CTTS().to(DEVICE),
    "tft": TemporalFusionTransformer(input_size=HPARAMS["input_size"]).to(DEVICE),
    "deepar": DeepAR(input_size=HPARAMS["input_size"]).to(DEVICE),
}

# Training function
def train_model(model, train_loader, criterion, optimizer, epochs=50):
    model.to(DEVICE)
    model.train()
    losses = []
    for epoch in range(epochs):
        epoch_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(train_loader)
        losses.append(avg_loss)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")
    return model, losses

# Train all models
save_path = os.path.join(PROJECT_ROOT, "saved_models/")
os.makedirs(save_path, exist_ok=True)

criterion = nn.MSELoss()
all_losses = {}

for name, model in models.items():
    print(f"\nTraining {name.upper()}...")
    optimizer = optim.Adam(model.parameters(), lr=HPARAMS["learning_rate"])
    trained_model, losses = train_model(model, train_loader, criterion, optimizer, HPARAMS["epochs"])
    torch.save(trained_model.state_dict(), os.path.join(save_path, f"{name}.pth"))
    all_losses[name] = losses
    print(f"✅ Model {name} saved.")

# Plot training losses
plt.figure(figsize=(10,5))
for name, losses in all_losses.items():
    plt.plot(losses, label=name)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Losses for Models")
plt.legend()
plt.show()

In [7]:
from utils.preprocessing import prepare_data
print("✅ Import successful!")


ImportError: cannot import name 'prepare_data' from 'utils.preprocessing' (/Users/jamesliu/Documents/GitHub/bloomberg-time-series/utils/preprocessing.py)